# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using GraphLab Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up graphlab create

In [1]:
import graphlab

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = graphlab.SFrame('kc_house_data.gl/')

This non-commercial license of GraphLab Create is assigned to davidshimer@yahoo.com and will expire on March 11, 2017. For commercial licensing options, visit https://dato.com/buy/.


2016-05-03 07:38:53,200 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1462286327.log


# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
            'bathrooms',
            'sqft_living', 'sqft_living_sqrt',
            'sqft_lot', 'sqft_lot_sqrt',
            'floors', 'floors_square',
            'waterfront', 'view', 'condition', 'grade',
            'sqft_above',
            'sqft_basement',
            'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in GraphLab Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = graphlab.linear_regression.create(sales, target='price', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.391090     | 6869667.895833     | 385998.458623 |

| 6         | 7        | 0.000002  | 1.436985     | 6847177.773672     | 380824.455891 |

+-----------+----------+-----------+--------------+--------------------+---------------+

TERMINATED: Iteration limit reached.

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [7]:
print model_all.get('coefficients').print_rows(num_rows=30)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None |  274873.05595 |  None  |
|     bedrooms     |  None |      0.0      |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 8468.53108691 |  None  |
|   sqft_living    |  None | 24.4207209824 |  None  |
| sqft_living_sqrt |  None | 350.060553386 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 842.068034898 |  None  |
|    sqft_above    |  None | 20.0247224171 |  None  |
|  sqft_basement   |  None |

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [8]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [15]:
import numpy as np
output = {}
for l1_penalty in np.logspace(1, 7, num=13):
    model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=l1_penalty)
    predictions = model.predict(validation)
    # Then compute the residuals/errors
    outcome = validation['price']
    residuals = outcome - predictions
    # Then square and add them up
    RSS = sum(residuals * residuals)
    output[l1_penalty] = RSS
print sorted(output.items())
print min(output, key=output.get)

best_model = graphlab.linear_regression.create(training, target='price', features=all_features,
                                              validation_set=None, verbose=False,
                                              l2_penalty=0., l1_penalty=10.0)
predictions = best_model.predict(testing)
# Then compute the residuals/errors
outcome = testing['price']
residuals = outcome - predictions
# Then square and add them up
RSS = sum(residuals * residuals)
print RSS

[(10.0, 625766285142461.2), (31.622776601683793, 625766285362395.2), (100.0, 625766286057887.0), (316.22776601683796, 625766288257224.9), (1000.0, 625766295212186.0), (3162.2776601683795, 625766317206077.8), (10000.0, 625766386760661.5), (31622.776601683792, 625766606749281.4), (100000.0, 625767302791633.4), (316227.76601683791, 625769507643885.1), (1000000.0, 625776517727025.8), (3162277.6601683795, 625799062845466.9), (10000000.0, 625883719085424.5)]
10.0
1.56983602382e+14


## looks like 10.0 wins
## 1.56983602382e+14

*** QUIZ QUESTIONS ***
1. What was the best value for the `l1_penalty`?
2. What is the RSS on TEST data of the model with the best `l1_penalty`?

In [17]:
print best_model.get('coefficients').print_rows(num_rows=30)

+------------------+-------+------------------+--------+
|       name       | index |      value       | stderr |
+------------------+-------+------------------+--------+
|   (intercept)    |  None |  18993.4272128   |  None  |
|     bedrooms     |  None |  7936.96767903   |  None  |
| bedrooms_square  |  None |  936.993368193   |  None  |
|    bathrooms     |  None |  25409.5889341   |  None  |
|   sqft_living    |  None |  39.1151363797   |  None  |
| sqft_living_sqrt |  None |  1124.65021281   |  None  |
|     sqft_lot     |  None | 0.00348361822299 |  None  |
|  sqft_lot_sqrt   |  None |  148.258391011   |  None  |
|      floors      |  None |   21204.335467   |  None  |
|  floors_square   |  None |  12915.5243361   |  None  |
|    waterfront    |  None |  601905.594545   |  None  |
|       view       |  None |  93312.8573119   |  None  |
|    condition     |  None |  6609.03571245   |  None  |
|      grade       |  None |  6206.93999188   |  None  |
|    sqft_above    |  None |  4

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

## looks like 18

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [18]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [19]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model['coefficients']['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [20]:
i = 0
model = []
nozero = []
rss = []

for l1penalty in l1_penalty_values:
    model.append(graphlab.linear_regression.create(training, target='price',
                                                  features=all_features,
                                                  l1_penalty=l1penalty,
                                                  l2_penalty=0.,
                                                  validation_set=None,
                                                  verbose=False))
    

    # get an array of "no zeros", that is, printing this array
    # will only print the # of features that have weights    

    #noz = model[i].get('coefficients')
    #noz = noz[noz['value']!=0]
    #nozero.append(len(noz))
    nozero.append(model[i]['coefficients']['value'].nnz())
    
    rss.append(((validation['price'] - model[i].predict(validation))**2).sum())
    
    print "Model " + str(i).rjust(3) + " => " + str(nozero[i]) + ' (' + str(l1penalty).rjust(20) + ') rss= ' + \
        str(rss[i]).rjust(20)
    
    i += 1

Model   0 => 18 (         100000000.0) rss=    6.27492659875e+14
Model   1 => 18 (        127427498.57) rss=    6.28210516771e+14
Model   2 => 18 (       162377673.919) rss=    6.29176689541e+14
Model   3 => 18 (       206913808.111) rss=    6.30650082719e+14
Model   4 => 17 (       263665089.873) rss=    6.32940229287e+14
Model   5 => 17 (       335981828.628) rss=     6.3626814023e+14
Model   6 => 17 (       428133239.872) rss=    6.41261198311e+14
Model   7 => 17 (       545559478.117) rss=    6.48983455376e+14
Model   8 => 17 (       695192796.178) rss=    6.60962217696e+14
Model   9 => 16 (        885866790.41) rss=    6.77261520728e+14
Model  10 => 15 (       1128837891.68) rss=    7.01046815867e+14
Model  11 => 15 (       1438449888.29) rss=    7.37850622829e+14
Model  12 => 13 (       1832980710.83) rss=     7.9616310964e+14
Model  13 => 12 (       2335721469.09) rss=    8.69018172894e+14
Model  14 => 10 (       2976351441.63) rss=    9.66925692362e+14
Model  15 => 6 (       37

Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzero` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzero` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [22]:
l1_penalty_min = 2976351441.63
l1_penalty_max = 3792690190.73 # or is it 4832930238.57

***QUIZ QUESTIONS***

What values did you find for `l1_penalty_min` and`l1_penalty_max`? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [23]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzero`.

In [24]:
i = 0
model = []
nozero = []
rss = []

for l1penalty in l1_penalty_values:
    model.append(graphlab.linear_regression.create(training, target='price',
                                                  features=all_features,
                                                  l1_penalty=l1penalty,
                                                  l2_penalty=0.,
                                                  validation_set=None,
                                                  verbose=False))
    

    # get an array of "no zeros", that is, printing this array
    # will only print the # of features that have weights    

    #noz = model[i].get('coefficients')
    #noz = noz[noz['value']!=0]
    #nozero.append(len(noz))
    nozero.append(model[i]['coefficients']['value'].nnz())
    
    rss.append(((validation['price'] - model[i].predict(validation))**2).sum())
    
    print "Model " + str(i).rjust(3) + " => " + str(nozero[i]) + ' (' + str(l1penalty).rjust(20) + ') rss= ' + \
        str(rss[i]).rjust(20)
    
    i += 1

Model   0 => 10 (       2976351441.63) rss=    9.66925692362e+14
Model   1 => 10 (       3019316638.95) rss=    9.74019450085e+14
Model   2 => 10 (       3062281836.27) rss=    9.81188367942e+14
Model   3 => 10 (       3105247033.59) rss=    9.89328342459e+14
Model   4 => 10 (       3148212230.91) rss=    9.98783211266e+14
Model   5 => 10 (       3191177428.24) rss=    1.00847716702e+15
Model   6 => 10 (       3234142625.56) rss=    1.01829878055e+15
Model   7 => 10 (       3277107822.88) rss=    1.02824799221e+15
Model   8 => 8 (        3320073020.2) rss=    1.03461690923e+15
Model   9 => 8 (       3363038217.52) rss=    1.03855473594e+15
Model  10 => 8 (       3406003414.84) rss=    1.04323723787e+15
Model  11 => 7 (       3448968612.16) rss=    1.04693748875e+15
Model  12 => 7 (       3491933809.48) rss=    1.05114762561e+15
Model  13 => 7 (        3534899006.8) rss=    1.05599273534e+15
Model  14 => 7 (       3577864204.12) rss=    1.06079953176e+15
Model  15 => 6 (       362082940

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [26]:
ourl1_penalty = 3448968612.16
print model[11]['coefficients']['value']


[222253.19254432785, 661.7227177822587, 0.0, 15873.957259267981, 32.41022145125964, 690.1147733133256, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2899.4202697498786, 30.011575302201045, 0.0, 0.0, 0.0]


In [27]:
print model[11].get('coefficients').print_rows(num_rows=30)

+------------------+-------+---------------+--------+
|       name       | index |     value     | stderr |
+------------------+-------+---------------+--------+
|   (intercept)    |  None | 222253.192544 |  None  |
|     bedrooms     |  None | 661.722717782 |  None  |
| bedrooms_square  |  None |      0.0      |  None  |
|    bathrooms     |  None | 15873.9572593 |  None  |
|   sqft_living    |  None | 32.4102214513 |  None  |
| sqft_living_sqrt |  None | 690.114773313 |  None  |
|     sqft_lot     |  None |      0.0      |  None  |
|  sqft_lot_sqrt   |  None |      0.0      |  None  |
|      floors      |  None |      0.0      |  None  |
|  floors_square   |  None |      0.0      |  None  |
|    waterfront    |  None |      0.0      |  None  |
|       view       |  None |      0.0      |  None  |
|    condition     |  None |      0.0      |  None  |
|      grade       |  None | 2899.42026975 |  None  |
|    sqft_above    |  None | 30.0115753022 |  None  |
|  sqft_basement   |  None |